# Fry Counter Grid Search Contours (FC_GSC_400)

In [1]:
import cv2 as cv
import numpy as np
import os

In [2]:
# Function for opening image
def get_img(img_path, grayscale=True):
    img = cv.imread(img_path)
    if grayscale:
        return cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return img

In [3]:
# Function for cropping image
def crop_img(img):
    img_centerX = img.shape[1] // 2
    img_centerY = img.shape[0] // 2

    # Initialize mask to use
    mask = np.zeros(img.shape, dtype=np.uint8)

    # Crop the image
    region_to_cut = cv.ellipse(mask, (img_centerX, img_centerY), (547, 547), 0, 0, 360, 255, -1)
    cropped_img = cv.bitwise_or(img, img, mask=region_to_cut)
    return cropped_img[:, img_centerX-600:img_centerX+600]

In [4]:
# Gaussian Blur Function
def gaussian_blur(img, kernel_a, kernel_b):
    return cv.GaussianBlur(img, (kernel_a, kernel_b), 0)

In [5]:
# Remove the background from the fry image
def img_diff(img, bg_img):
    return cv.absdiff(img, bg_img)

In [6]:
# Image Segmentation Using Thresholding Function
def threshold_img(diff_img, type='OTSU', block=0, c=0):
    if type == 'OTSU':
        _, thresh_img = cv.threshold(diff_img, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
        return thresh_img
    else:
        return cv.adaptiveThreshold(diff_img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_OTSU, block, c)

In [7]:
# Histogram Counter
def histogram_counter(thres_img, pixel_to_count='white'):
    if pixel_to_count == 'white':
        return np.count_nonzero(thres_img == 255)
    else:
        return np.count_nonzero(thres_img == 0)

In [8]:
# Countour Detection
def contour_detection(thres_img):
    contours, _ = cv.findContours(thres_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    return contours

# GRID SEARCH

In [9]:
# Path to the images (400)
path = '../IMAGES/400/'
list_images_400 = [f for f in os.listdir(path) if f.startswith('my_photo')]
list_paths_400 = [path + f for f in list_images_400]

In [10]:
# Kernel values for Grid Searching

# Kernel values for Fry Image
kernel_values_a = [3, 5, 7, 9, 11, 13]
kernel_values_b = [3, 5, 7, 9, 11, 13]

# Kernel values for Background Image
kernel_values_c = [3, 5, 7, 9, 11, 13]
kernel_values_d = [3, 5, 7, 9, 11, 13]

In [11]:
# Initialize the best accuracy (FOR 400 FRIES)
best_accuracy_400 = None
best_400_a = None
best_400_b = None
best_400_c = None
best_400_d = None

for a in kernel_values_a:
    for b in kernel_values_b:
        for c in kernel_values_c:
            for d in kernel_values_d:
                contours_val = np.array([])
                for img_path in list_paths_400:
                    img = get_img(img_path)
                    bg_img = get_img('../IMAGES/400/background.jpg')

                    # Crop the image
                    img = crop_img(img)
                    bg_img = crop_img(bg_img)

                    # Gaussian Blur
                    img = gaussian_blur(img, a, b)
                    bg_img = gaussian_blur(bg_img, c, d)

                    # Image Difference
                    diff_img = img_diff(img, bg_img)

                    # OTSU Thresholding
                    thres_img = threshold_img(diff_img)

                    # Contour Detection
                    contours = contour_detection(thres_img)
                    contours_val = np.append(contours_val, len(contours))
                # Accuracy measures how close is the mean contours to 400
                diff_to_400 = np.abs(np.mean(contours_val) - 400)
                accuracy = ((400 - diff_to_400) / 400 ) * 100
                if best_accuracy_400 is None or accuracy > best_accuracy_400:
                    best_accuracy_400 = accuracy
                    best_400_a = a
                    best_400_b = b
                    best_400_c = c
                    best_400_d = d

print(f"Best Accuracy for 400 images: {best_accuracy_400} \nBest Kernel for Fry Images: ({best_400_a}, {best_400_b}) \nBest Kernel for Background Images: ({best_400_c}, {best_400_d})")

Best Accuracy for 400 images: 99.935 
Best Kernel for Fry Images: (11, 11) 
Best Kernel for Background Images: (9, 3)
